In [1]:
import sys
sys.path.append('..')

In [ ]:
import random
import numpy as np
from library.solution import Solution
from library.problems.tsp import TSPSolution
from library.problems.ks import KSSolution

## Simulated Annealing

Simulated Annealing is an optimization algorithm that explores solutions by allowing both improvements and occasional worse moves to escape local optima. The probability of accepting worse solutions decreases over time, controlled by a temperature parameter that gradually cools. This balance between exploration and exploitation helps the algorithm find a global optimum rather than getting stuck in suboptimal solutions.

### Pseudo-code

1. Define the current solution (usually at random)
2. Repeat until termination condition (usually nr of iterations):
    1. Repeat **L** times:
        1. Choose a random neighbor of the current solution
        2. If random neighbor is better than current solution, replace current solution by neighbor. Otherwise, accept the nieghbor as the current solution with probability: $$exp(-\frac{|neighbor.fitness - current.fitness|}{C})$$
    2. Decrement **C** by dividing it by **H**
3. Return current solution

### Algorithm Implementation

Let's implement the simmulated annealing algorithm using python. The function that implements the algorithm should receive the following arguments:
- `initial_solution`: Initial current solution
- `C`: Control parameter
- `L`: Number of iterations with same C
- `H`: Decreasing rate of parameter C
- `maximization`: boolean that indicates if we're solving a maximization or minimization problem
- `max_iter`: maximum number of interations.

In [3]:
from copy import deepcopy

def simulated_annealing(
    initial_solution: Solution,
    C: float = 100,
    L: int = 10,
    H: float = 1.1,
    maximization: bool = True,
    max_iter: int | None = 10,
    verbose: bool = False,
):
    # 1. Initialize solution
    current_solution = initial_solution

    iter = 1

    if verbose:
        print(f'Initial solution: {current_solution.repr} with fitness {current_solution.fitness()}')

    # 2. Repeat until termination condition
    while iter <= max_iter:
    
        # 2.1 For L times
        for _ in range(L):
            # 2.1.1 Get random neighbor
            random_neighbor = current_solution.get_random_neighbor()

            neighbor_fitness = random_neighbor.fitness()
            current_fitness = current_solution.fitness()

            if verbose:
                print(f"Random neighbor {random_neighbor} with fitness: {neighbor_fitness}")

            # 2.1.2 Decide if neighbor is accepted as new solution
            # If neighbor is better, accept it
            if (
                (maximization and (neighbor_fitness >= current_fitness))
                or(not maximization and (neighbor_fitness <= current_fitness))
            ):
                current_solution = deepcopy(random_neighbor)
                if verbose:
                    print(f'Neighbor is better. Replaced current solution by neighbor.')

            # If neighbor is worse, accept it with a certain probability
            # Maximizaton: Neighbor is worse than current solution if fitness is lower
            # Minimization: Neighbor is worse than current solution if fitness is higher
            elif (
                (maximization and (neighbor_fitness < current_fitness)
                 or (not maximization and (neighbor_fitness > current_fitness)))
            ):
                # Generate random number between 0 and 1
                random_float = random.random()
                # Define probability P
                p = np.exp(-abs(current_fitness - neighbor_fitness) / C)
                if verbose:
                    print(f'Probability of accepting worse neighbor: {p}')
                # The event happens with probability P if the random number if lower than P
                if random_float < p:
                    current_solution = deepcopy(random_neighbor)
                    if verbose:
                        print(f'Neighbor is worse and was accepted.')
                else:
                    if verbose:
                        print("Neighbor is worse and was not accepted.")

            if verbose:
                print(f"New current solution {current_solution} with fitness {current_solution.fitness()}")

        # 2.2 Update C
        C = C / H
        if verbose:
            print(f'Decreased C. New value: {C}')
            print('--------------')

        iter += 1

    if verbose:
        print(f'Best solution found: {current_solution.repr} with fitness {current_solution.fitness()}')
    
    # 3. Return solution
    return current_solution

Notice that we assume that a solution has the following methods:
- `fitness()`
- `get_random_neighbor()`

Additionally, `get_random_neighbor()` must return a solution that also implements these methods.

### Solving TSP with Simulated Annealing

To solve TSP with simulated annealing we need to define a `TSPSASolution` class that inherits from `TSPSolution` and implements the `get_random_neighbor()` method.

In the previous notebook, we implemented `TSPSolution`, which provides the `fitness()` and `random_initial_value()` methods. We also created `TSPHillClimbingSolution`, which extends `TSPSolution` and implements `get_neighbors()`.

Simulated Annealing requires selecting only random neighbor rather than generating all neighbors. Therefore, we can create a new class `TSPSASolution`, that implements the method that is required for simulated annealing to work: `get_random_neighbor()`.

We could do this two ways:
- Inherit from `TSPHillClimbingSolution` and use the `get_neighbors()` method inside the `get_random_neighbor()` method to first get all neighbors, and then radomly select one
- Inherit from `TSPSolution` and implement only the `get_random_neighbor()`

Let's go with the second one to keep the code as independent, eficient and modular as possible.

![TSP Solutions](images/tsp-solutions.png)

A neighbor of a TSP solution can be obtained by swapping two consecutive cities on the route (excluding the starting and end points).

In [4]:
class TSPSASolution(TSPSolution):
    def get_random_neighbor(self):
        nr_cities = len(self.distance_matrix)

        # Choose a city idx to switch with the next city 
        random_city_idx = random.randint(1, nr_cities-3)

        new_route = deepcopy(self.repr)
        new_route[random_city_idx] = self.repr[random_city_idx+1]
        new_route[random_city_idx+1] = self.repr[random_city_idx]

        return TSPSASolution(repr=new_route, distance_matrix=self.distance_matrix, starting_idx=self.starting_idx)

Let's test it

In [5]:
solution = TSPSASolution()

print('Solution', solution)
print('Random neighbor', solution.get_random_neighbor())

Solution [0, 8, 7, 12, 1, 2, 5, 6, 10, 11, 4, 9, 3, 0]
Random neighbor [0, 8, 7, 12, 1, 2, 5, 10, 6, 11, 4, 9, 3, 0]


And now we can apply the simulated annealing algorithm by giving it an random initial solution

In [ ]:
initial_solution = TSPSASolution()
simulated_annealing(initial_solution, maximization=False, verbose=True)

The implementation of `TSPSASolution` can be found in `library/problems/tsp.py`

### Solving KS with Simulated Annealing

To solve Knapsack with simulated annealing we need to define a `KSSASolution` class that inherits from `KSSolution` and implements the `get_random_neighbor()` method.

In the previous notebook, we implemented `KSSolution`, which provides the `fitness()` and `random_initial_value()` methods. We also created `KSHillClimbingSolution`, which extends `KSSolution` and implements `get_neighbors()`.

Since Simulated Annealing requires selecting a random neighbor rather than generating all neighbors, we can create a new class, `KSSASolution`, that implements the `get_random_neighbor()` method.

Similarly to what we just did for TSP, let's implement the `KSSASolution` that inherits from `TSPSolution` and implements the `get_random_neighbor()`.

A neighbor of a KS solution can be obtained by randomly flipping a bit, meaning, adding or removing an item from the knapsack.

In [ ]:
class KSSASolution(KSSolution):
    def get_random_neighbor(self):
        neighbor_repr = deepcopy(self.repr)
        # Get random index
        random_idx = random.randint(0, len(self.values)-1)
        # Bit flip
        if neighbor_repr[random_idx] == 1:
            neighbor_repr[random_idx] = 0
        else:
            neighbor_repr[random_idx] = 1
        
        return KSSASolution(
            repr=neighbor_repr,
            values=self.values,
            weights=self.weights,
            capacity=self.capacity,
        )

Let's test it

In [8]:
solution = KSSASolution()

print('Solution', solution)
print('Random neighbor', solution.get_random_neighbor())

Solution [1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1]
Random neighbor [1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1]


And now we can apply the simulated annealing algorithm by giving it an random initial solution

In [ ]:
initial_solution = KSSASolution()
simulated_annealing(initial_solution, maximization=True, max_iter=10, verbose=True)